In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pprint
import scipy
import seaborn as sns
import string
from matplotlib.patches import Patch
from sklearn.model_selection import (
    GroupKFold,
    GroupShuffleSplit,
    KFold,
    ShuffleSplit,
    StratifiedKFold,
    StratifiedShuffleSplit,
    TimeSeriesSplit,
)

pp = pprint.PrettyPrinter(indent=4)

print("Using pandas %s version" % pd.__version__)
print("Using seaborn %s version" % sns.__version__)
print("Using scipy %s version" % scipy.__version__)

Using pandas 1.2.0 version
Using seaborn 0.11.1 version
Using scipy 1.6.0 version


In [2]:
movies_names = ["MovieID", "Title", "Genres"]

movies = pd.read_table(
    r"C:\Projects\RecSys2020\datasets\ml1m\movies.dat",
    sep="::",
    header=None,
    names=movies_names,
    encoding="latin-1",
    engine="python",
)

movies.head()

ratings_names = ["UserID", "MovieID", "Rating", "Timestamp"]

ratings = pd.read_table(
    r"C:\Projects\RecSys2020\datasets\ml1m\ratings.dat",
    sep="::",
    header=None,
    engine="python",
    names=ratings_names,
)

ratings.head()

users_names = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]

users = pd.read_table(
    r"C:\Projects\RecSys2020\datasets\ml1m\users.dat",
    sep="::",
    engine="python",
    header=None,
    names=users_names,
)

In [3]:
# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

In [10]:
lens.head()

# lens.describe()

,MovieID,Title,Genres,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067


In [17]:
lens['PositiveRating'] = np.where(lens['Rating'] >= 4, True, False)
lens['PR'] = np.where(lens['Rating'] >= 4, 1, 0)
lens

,MovieID,Title,Genres,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,PositiveRating,PR
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268,F,1,10,48067,True,1
1,48,Pocahontas (1995),Animation|Children's|Musical|Romance,1,5,978824351,F,1,10,48067,True,1
2,150,Apollo 13 (1995),Drama,1,5,978301777,F,1,10,48067,True,1
3,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1,4,978300760,F,1,10,48067,True,1
4,527,Schindler's List (1993),Drama|War,1,5,978824195,F,1,10,48067,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,3513,Rules of Engagement (2000),Drama|Thriller,5727,4,958489970,M,25,4,92843,True,1
1000205,3535,American Psycho (2000),Comedy|Horror|Thriller,5727,2,958489970,M,25,4,92843,False,0
1000206,3536,Keeping the Faith (2000),Comedy|Romance,5727,5,958489902,M,25,4,92843,True,1
1000207,3555,U-571 (2000),Action|Thriller,5727,3,958490699,M,25,4,92843,False,0


In [9]:
# lens.groupby("MovieID")["Title"] #.agg({"Rating": pd.Series.count})
lens.groupby(["MovieID"])["Title"] \
    .count() \
    .reset_index(name='count') \
    .sort_values(['count'], ascending=False) \
    .head(5)

,MovieID,count
2651,2858,3428
253,260,2991
1106,1196,2990
1120,1210,2883
466,480,2672


In [7]:
lens22 = lens[['MovieID','Title']] \
                             .groupby(['MovieID'])['Title'] \
                             .size() \
                             .nlargest(5) \
                             .reset_index(name='top5')

lens22

,MovieID,top5
0,2858,3428
1,260,2991
2,1196,2990
3,1210,2883
4,480,2672


,MovieID,top5
0,2858,3428
1,260,2991
2,1196,2990
3,1210,2883
4,480,2672


In [19]:
# https://analyticsindiamag.com/how-to-build-your-first-recommender-system-using-python-movielens-dataset/
Average_ratings = pd.DataFrame(lens.groupby('Title')['Rating'].mean())
Average_ratings.head(10)

,Rating
Title,
"$1,000,000 Duck (1971)",3.027027
'Night Mother (1986),3.371429
'Til There Was You (1997),2.692308
"'burbs, The (1989)",2.910891
...And Justice for All (1979),3.713568
1-900 (1994),2.500000
10 Things I Hate About You (1999),3.422857
101 Dalmatians (1961),3.596460
101 Dalmatians (1996),3.046703


In [23]:
Average_ratings['Total Ratings'] = pd.DataFrame(lens.groupby('Title')['Rating'].count())
Average_ratings['Total Positive Ratings'] = pd.DataFrame(lens.groupby('Title')['PR'].sum())
# Average_ratings.sort_values('Total Ratings',ascending=False).reset_index()
Average_ratings.sort_values(['Rating', 'Total Ratings'],ascending=[False, False]).reset_index()
# sort_values(['a', 'b'], ascending=[True, False])
# Average_ratings.head(10)

,Title,Rating,Total Ratings,Total Positive Ratings
0,"Gate of Heavenly Peace, The (1995)",5.0,3,3
1,Smashing Time (1967),5.0,2,2
2,"Baby, The (1973)",5.0,1,1
3,Bittersweet Motel (2000),5.0,1,1
4,Follow the Bitch (1998),5.0,1,1
...,...,...,...,...
3701,Underworld (1997),1.0,1,0
3702,"Uninvited Guest, An (2000)",1.0,1,0
3703,Waltzes from Vienna (1933),1.0,1,0
3704,White Boys (1999),1.0,1,0


In [25]:
Average_ratings.sort_values('Total Positive Ratings',ascending=False).reset_index().head(50)

,Title,Rating,Total Ratings,Total Positive Ratings
0,American Beauty (1999),4.317386,3428,2853
1,Star Wars: Episode IV - A New Hope (1977),4.453694,2991,2622
2,Star Wars: Episode V - The Empire Strikes Back...,4.292977,2990,2510
3,Saving Private Ryan (1998),4.337354,2653,2260
4,Raiders of the Lost Ark (1981),4.477725,2514,2260
5,"Silence of the Lambs, The (1991)",4.351823,2578,2252
6,"Matrix, The (1999)",4.315830,2590,2171
7,"Sixth Sense, The (1999)",4.406263,2459,2163
8,Star Wars: Episode VI - Return of the Jedi (1983),4.022893,2883,2127
9,Fargo (1996),4.254676,2513,2074


In [ ]:
# Age
rating_by_age = lens.groupby("Age").size()
users_by_age = lens.groupby("Age").agg({"UserID": pd.Series.nunique})


# pp.pprint({'number of ratings by age':rating_by, 'unique users by age':users_by})


def groupby(by):
    r = lens.groupby(by).size()
    u = lens.groupby(by).agg({"UserID": pd.Series.nunique})
    pp.pprint({"number of ratings by " + by: r, "unique users by " + by: u})


for i in ["Gender", "Age", "Occupation"]:
    groupby(i)

In [63]:
rating_by_age

Age
1      27211
18    183536
25    395556
35    199003
45     83633
50     72490
56     38780
dtype: int64

In [5]:
age = {
    1: "Under 18",
    18: "18-24",
    25: "25-34",
    35: "35-44",
    45: "45-49",
    50: "50-55",
    56: "56+",
}

pd.DataFrame(
    data=[rating_by_age.tolist(), np.array(users_by_age)],
    index=["ratings", "users"],
    columns=age.values(),
)

,Under 18,18-24,25-34,35-44,45-49,50-55,56+
ratings,27211,183536,395556,199003,83633,72490,38780
users,[222],[1103],[2096],[1193],[550],[496],[380]


In [6]:
gender = {"F": "Female", "M": "Male"}
age = {
    1: "Under 18",
    18: "18-24",
    25: "25-34",
    35: "35-44",
    45: "45-49",
    50: "50-55",
    56: "56+",
}
occupation = {
    0: "other or not specified",
    1: "academic/educator",
    2: "artist",
    3: "clerical/admin",
    4: "college/grad student",
    5: "customer service",
    6: "doctor/health care",
    7: "executive/managerial",
    8: "farmer",
    9: "homemaker",
    10: "K-12 student",
    11: "lawyer",
    12: "programmer",
    13: "retired",
    14: "sales/marketing",
    15: "scientist",
    16: "self-employed",
    17: "technician/engineer",
    18: "tradesman/craftsman",
    19: "unemployed",
    20: "writer",
}
names = {"Gender": gender, "Age": age, "Occupation": occupation}


def getNames(by: string):
    return names[by]


def groupby2(by):
    r = lens.groupby(by).size()
    u = lens.groupby(by).agg({"UserID": pd.Series.nunique})
    # pp.pprint({'number of ratings by '+ by:r, 'unique users by '+by:u})
    ppp = pd.DataFrame(
        data=[r.tolist(), np.array(u)],
        index=["ratings", "users"],
        columns=getNames(by).values(),
    )
    return ppp


e = []
for i in ["Gender", "Age", "Occupation"]:
    e.append(groupby2(i))

In [7]:
e[0].T

,ratings,users
Female,246440,[1709]
Male,753769,[4331]


In [ ]:
# gender = {'M': 'Male', 'F': 'Female'}
# age = {
#     1:"Under 18",
#    18:"18-24",
#    25:"25-34",
#    35:"35-44",
#    45:"45-49",
#    50:"50-55",
#    56:"56+"}
# occupation = {0:"other or not specified",
#     1:"academic/educator",
#     2:"artist",
#     3:"clerical/admin",
#     4:"college/grad student",
#     5:"customer service",
#     6:"doctor/health care",
#     7:"executive/managerial",
#     8:"farmer",
#     9:"homemaker",
#    10:"K-12 student",
#    11:"lawyer",
#    12:"programmer",
#    13:"retired",
#    14:"sales/marketing",
#    15:"scientist",
#    16:"self-employed",
#    17:"technician/engineer",
#    18:"tradesman/craftsman",
#    19:"unemployed",
#    20:"writer"}
# names = {"Gender":gender, "Age":age, "Occupation":occupation}
# def getNames(by: string):
#    return names[by]

# def groupby2(by):
#    r = lens.groupby(by).size()
#    u = lens.groupby(by).agg({"UserID": pd.Series.nunique})
#    u = u.replace({by:getNames(by)})
#    pp.pprint({'number of ratings by '+ by:r, 'unique users by '+by:u})

# for i in ["Gender", "Age", "Occupation"]:
#    groupby2(i)


# print(rating_by_age.axes)
# print(rating_by_age)
# print(rating_by_age.replace({'Age':age}))

# rating_by_age.reindex(["Under 18",
# "18-24",
# "25-34",
# "35-44",
# "45-49",
# "50-55",
# "56+"])#

In [37]:
data_dirM = r"C:\Projects\RecSys2020\datasets\ml1m\male\\"
data_dirF = r"C:\Projects\RecSys2020\datasets\ml1m\female\\"

fr = lens[lens["Gender"] == "F"].sort_values(by="UserID")
mr = lens[lens["Gender"] == "M"].sort_values(by="UserID")

rating_columns = ["UserID", "MovieID", "Rating", "Timestamp"]
fname = "data.txt"
params = dict(index=False, header=False, columns=rating_columns, sep="\t")
fr.to_csv(os.path.join(data_dirF, fname), **params)
mr.to_csv(os.path.join(data_dirM, fname), **params)

In [8]:
def save_csv(data):
    values = data["lens"].sort_values(by="UserID")

    try:
        os.mkdir(data["path"])
    except OSError:
        print("Creation of the directory %s failed" % data["path"])
    else:
        print("Successfully created the directory %s " % data["path"])

    csv_path = os.path.join(data["path"], "data.txt")
    values.to_csv(
        csv_path,
        index=False,
        header=False,
        columns=["UserID", "MovieID", "Rating", "Timestamp"],
        sep="\t",
    )


young = lens[lens["Age"] == 1]
path = os.path.join(r"C:\Projects\RecSys2020\datasets\ml1m\\", "young")
save_csv({"lens": young, "path": path})

old = lens[lens["Age"] == 56]
path = os.path.join(r"C:\Projects\RecSys2020\datasets\ml1m\\", "old")
save_csv({"lens": old, "path": path})


Successfully created the directory C:\Projects\RecSys2020\datasets\ml1m\\young 
Successfully created the directory C:\Projects\RecSys2020\datasets\ml1m\\old 


In [16]:
young.describe()
young['MovieID'].unique().size


2650

In [17]:
old.describe()
old['MovieID'].unique().size

2913

In [5]:
# Generate the class/group data
n_points = len(lens)
lens["sexcode"] = pd.factorize(lens["Gender"])[0] + 1
ageGroups = pd.factorize(lens["Age"])[0] + 1
occupationGroups = pd.factorize(lens["Occupation"])[0] + 1
timestampGroups = pd.factorize(lens["Timestamp"])[0] + 1

X = np.array(lens["Rating"])  # np.random.randn(100, 10)
y = np.array(lens["sexcode"]).T

n_splits = 5

np.random.seed(1338)
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

In [17]:
def plot_cv_indices(cv, X, y, group, group2, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    )

    ax.scatter(
        range(len(X)), [ii + 3.5] * len(X), c=group2, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    labels = ["Lytis", "Amžius", "Užsiėmimas"]
    yticklabels = list(range(n_splits)) + labels
    ax.set(
        yticks=np.arange(n_splits + 3) + 0.5,
        yticklabels=yticklabels,
        xlabel="Imties indeksas",
        ylabel="Kryžminės patikros padalijimas",
        ylim=[n_splits + len(labels) + 0.2, -0.2],
        xlim=[0, n_points],
    )
    ax.set_title(type(cv).__name__, fontsize=12)
    return ax


# cvs = [KFold]
# ,GroupKFold, ShuffleSplit, StratifiedKFold,
# GroupShuffleSplit, StratifiedShuffleSplit, TimeSeriesSplit]


cvs1 = [KFold, GroupKFold, ShuffleSplit]
cvs2 = [
    StratifiedKFold,
    ##GroupShuffleSplit,
    StratifiedShuffleSplit,
    TimeSeriesSplit,
]

fig, axs = plt.subplots(2, 3)

for i, cv in enumerate(cvs1):
    print(cv, i)
    this_cv = cv(n_splits=n_splits)
    ax = axs[0, i]
    plot_cv_indices(this_cv, X, y, ageGroups, occupationGroups, ax, n_splits)

for i, cv in enumerate(cvs2):
    print(cv, i)
    this_cv = cv(n_splits=n_splits)
    ax = axs[1, i]
    plot_cv_indices(this_cv, X, y, ageGroups, occupationGroups, ax, n_splits)

ax.legend(
    [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
    ["Testavimo aibė", "Mokymo aibė"],
    loc=(1.02, 0.8),
)
# Make the legend fit
fig.set_size_inches(12, 6)

plt.tight_layout()
png = os.path.join(r"C:\Projects\RecSys2020\results\figure0.cross-validations.png")
fig.savefig(png, format="png", dpi=300)

png = os.path.join(
    r"E:\OneDrive\MSThesis\_2019\master_thesis_template_vu_mif_cs1-master\master_thesis_template_vu_mif_cs1-master\_third_part\img\figure0.cross-validations.png"
)
fig.savefig(png, format="png", dpi=300)

# fig.subplots_adjust(right=0.7)
# plt.show()

<class 'sklearn.model_selection._split.KFold'> 0
<class 'sklearn.model_selection._split.GroupKFold'> 1
<class 'sklearn.model_selection._split.ShuffleSplit'> 2
<class 'sklearn.model_selection._split.StratifiedKFold'> 0
<class 'sklearn.model_selection._split.StratifiedShuffleSplit'> 1
<class 'sklearn.model_selection._split.TimeSeriesSplit'> 2


In [7]:
cv = TimeSeriesSplit(n_splits)
splits = []
for ii, (tr, tt) in enumerate(cv.split(X=lens, y=y, groups=timestampGroups)):
    splits.append((tr, tt))

print(len(splits[0][1]))

for i in splits:
    print(len(i[0]))

166701
166704
333405
500106
666807
833508


In [8]:
print([[len(t) for t in split] for split in splits])
splited = [[t for t in split] for split in splits]

[[166704, 166701], [333405, 166701], [500106, 166701], [666807, 166701], [833508, 166701]]


In [9]:
for split in splits:
    print(len(split))
# 1-data-test.txt
# 1-data-train.txt

2
2
2
2
2


In [6]:
data_dir = r"C:\Projects\RecSys2020\datasets\ml1m\\"
cvs = [
    KFold,
    GroupKFold,
    ShuffleSplit,
    StratifiedKFold,
    GroupShuffleSplit,
    StratifiedShuffleSplit,
    TimeSeriesSplit,
]

In [ ]:
cv = TimeSeriesSplit(n_splits)
for fold, (train, test) in enumerate(cv.split(X=lens, y=y, groups=timestampGroups)):
    fold += 1
    print(fold)
    print(fold, " train=", len(lens[lens.index.isin(splits[i][0])]))
    print(fold, " test=", len(lens[lens.index.isin(splits[i][1])]))

    train = lens[["UserID", "MovieID", "Rating", "Timestamp"]][lens.index.isin(train)]
    test = lens[["UserID", "MovieID", "Rating", "Timestamp"]][lens.index.isin(test)]
    cv_name = type(cv).__name__
    dirr = os.path.join(data_dir, cv_name)

    train.to_csv(os.path.join(dirr, str(fold) + "-data-train.txt"))
    test.to_csv(os.path.join(dirr, str(fold) + "-data-test.txt"))

In [43]:
lens[["UserID", "MovieID", "Rating", "Timestamp"]][lens.index.isin(splits[i][0])]

,UserID,MovieID,Rating,Timestamp
1,1,48,5,978824351
2,1,150,5,978301777


In [17]:
def split(cvs, groups):
    for cv in cvs:
        this_cv = cv(n_splits=n_splits)
        this_cv_name = type(this_cv).__name__
        dir = os.path.join(data_dir, this_cv_name)
        if not os.path.exists(dir):
            os.mkdir(dir)

        for fold, (train, test) in enumerate(this_cv.split(X=lens, y=y, groups=groups)):
            fold += 1
            print()
            print(this_cv_name, fold, " train=", len(lens[lens.index.isin(train)]))
            print(this_cv_name, fold, " test=", len(lens[lens.index.isin(test)]))

            train = lens[["UserID", "MovieID", "Rating", "Timestamp"]][
                lens.index.isin(train)
            ]
            test = lens[["UserID", "MovieID", "Rating", "Timestamp"]][
                lens.index.isin(test)
            ]

            train.to_csv(
                os.path.join(dir, str(fold) + "-data-train.txt"),
                index=False,
                header=False,
                sep="\t",
            )
            test.to_csv(
                os.path.join(dir, str(fold) + "-data-test.txt"),
                index=False,
                header=False,
                sep="\t",
            )

In [19]:
def split(cvs, groups):
    for cv in cvs:
        this_cv = cv(n_splits=n_splits)
        this_cv_name = type(this_cv).__name__
        dir = os.path.join(data_dir, this_cv_name)
        if not os.path.exists(dir):
            os.mkdir(dir)

        for fold, (train, test) in enumerate(this_cv.split(X=lens, y=y, groups=groups)):
            fold += 1
            print()
            print(this_cv_name, fold, " train=", len(lens[lens.index.isin(train)]))
            print(this_cv_name, fold, " test=", len(lens[lens.index.isin(test)]))

            train = lens[["UserID", "MovieID", "Rating", "Timestamp"]][
                lens.index.isin(train)
            ]
            test = lens[["UserID", "MovieID", "Rating", "Timestamp"]][
                lens.index.isin(test)
            ]

            train.to_csv(
                os.path.join(dir, str(fold) + "-data-train.txt"),
                index=False,
                header=False,
                sep="\t",
            )
            test.to_csv(
                os.path.join(dir, str(fold) + "-data-test.txt"),
                index=False,
                header=False,
                sep="\t",
            )


data_dir = r"C:\Projects\RecSys2020\datasets\ml1m\\"
cvs = [KFold, ShuffleSplit, StratifiedKFold, GroupShuffleSplit, StratifiedShuffleSplit]
split(cvs, np.array(lens["sexcode"]).T)
split([TimeSeriesSplit], timestampGroups)
# , TimeSeriesSplit, GroupKFold]
# g = np.array(lens['sexcode']).T


KFold 1  train= 800167
KFold 1  test= 200042

KFold 2  train= 800167
KFold 2  test= 200042

KFold 3  train= 800167
KFold 3  test= 200042

KFold 4  train= 800167
KFold 4  test= 200042

KFold 5  train= 800168
KFold 5  test= 200041

ShuffleSplit 1  train= 900188
ShuffleSplit 1  test= 100021

ShuffleSplit 2  train= 900188
ShuffleSplit 2  test= 100021

ShuffleSplit 3  train= 900188
ShuffleSplit 3  test= 100021

ShuffleSplit 4  train= 900188
ShuffleSplit 4  test= 100021

ShuffleSplit 5  train= 900188
ShuffleSplit 5  test= 100021

StratifiedKFold 1  train= 800167
StratifiedKFold 1  test= 200042

StratifiedKFold 2  train= 800167
StratifiedKFold 2  test= 200042

StratifiedKFold 3  train= 800167
StratifiedKFold 3  test= 200042

StratifiedKFold 4  train= 800167
StratifiedKFold 4  test= 200042

StratifiedKFold 5  train= 800168
StratifiedKFold 5  test= 200041

GroupShuffleSplit 1  train= 753769
GroupShuffleSplit 1  test= 246440

GroupShuffleSplit 2  train= 753769
GroupShuffleSplit 2  test= 246440
